# Convert .mat files to .csv files (both trn & inf)

## Manual:
After unzipping, for example, Tail_687_3
* make subdir Tail_687_3/hold (move all .mat files here)
* make subdir Tail_687_3/1min (all converted .csv files will be put here)
* set TAIL = 'Tail_687_3'
* move all .csv files to Train/1min or Test/1min


In [0]:
import scipy.io
import numpy as np
import pandas as pd
from functools import reduce
import glob
from datetime import datetime
from IPython.core.debugger import set_trace

In [0]:
!pwd

In [0]:
def remove_unused_keys(matlab_dict):
    '''#Remove unused keys from matlab dictionary.'''
    del mat['__header__']
    del mat['__version__']
    del mat['__globals__']
    return matlab_dict

In [0]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [0]:
def get_columns_for_rate(rate, matlab_dict):
    '''Returns sorted list of all variables with a given sample rate.'''
    #set_trace()
    vars = []
    rates = []
    for key, value in matlab_dict.items():
        vars.append(key)
        rates.append(matlab_dict[key][0][0][1][0][0])
    #print(vars); print(rates)
    vars_df = pd.DataFrame({'var': vars, 'rate': rates})
    #vars_df.set_index('var', inplace=True)
    candidates = vars_df.loc[vars_df.rate==rate, :]
    sorted_candidates = np.sort(candidates['var'].values)#; print('number of candidates=', len(candidates))
    return sorted_candidates

In [0]:
def get_dfs_for_rate(rate, matlab_dict):
    '''Return list of dataframes with given rate.'''
    r = [] #rate_list
    rate_cols = get_columns_for_rate(rate, matlab_dict)
    for i,val in enumerate(rate_cols):
        #print(val)
        data = np.squeeze(matlab_dict[val][0][0][0])
        r.append(pd.DataFrame({val: data}))
    return r

In [0]:
!ls

10_mat2csv-2.ipynb  20_eda1-2.ipynb  30_mod1.ipynb  Tail_687_1	Tail_687_3
10_mat2csv.ipynb    20_eda1.ipynb    30_mod2.ipynb  Tail_687_2	Tail_test


In [0]:
#TAIL = 'Tail_687_1'
#TAIL = 'Tail_687_2'
TAIL = 'Tail_687_3'
HIGEST_RATE = 16

In [0]:
! pwd

/home/proj/DASHlink


In [0]:
#ensure no output files, i.e. -1min.csv
#! rm {TAIL}/*.csv
! ls {TAIL}/*.csv

ls: cannot access 'Tail_687_3/*.csv': No such file or directory


In [0]:
#cp all input files (.mat) from hold to tail dir
! cp {TAIL}/hold/*.mat {TAIL} 

In [0]:
! ls {TAIL}/*.*

In [0]:
allFiles = glob.glob(TAIL + "/*.mat")
n = len(allFiles)
print(f'TOTAL FILES TO PROCESS: {n}\n')
for i,file_ in enumerate(allFiles):
    #read matlab file into mat
    mat = scipy.io.loadmat(file_); print(file_) 
    #remove unused keys from matlab dict
    mat = remove_unused_keys(mat)
    
    # All dataframes for each rate
    #print('rate=0.25')
    dfs = get_dfs_for_rate(0.25, mat)#; print('rate=0.25')
    dfp25 = reduce((lambda x, y: x.join(y)), dfs)#; print(dfp25.shape)
    repeats = round(HIGEST_RATE/0.25)#; print('repeats =', repeats)
    dfp25 = dfp25.loc[np.repeat(dfp25.index.values, repeats)]#; print(len(dfp25))
    dfp25 = dfp25.reset_index(drop=True)

    #print('rate=1')
    dfs = get_dfs_for_rate(1, mat)
    df1 = reduce((lambda x, y: x.join(y)), dfs)#; print(df1.shape)
    repeats = round(HIGEST_RATE/1)#; print('repeats =', repeats)
    df1 = df1.loc[np.repeat(df1.index.values, repeats)]#; print(len(df1))
    df1 = df1.reset_index(drop=True)

    #print('rate=2')
    dfs = get_dfs_for_rate(2, mat)
    df2 = reduce((lambda x, y: x.join(y)), dfs)#; print(df2.shape)
    repeats = round(HIGEST_RATE/2)#; print('repeats =', repeats)
    df2 = df2.loc[np.repeat(df2.index.values, repeats)]#; print(len(df2))
    df2 = df2.reset_index(drop=True)

    #print('rate=4')
    dfs = get_dfs_for_rate(4, mat)
    df4 = reduce((lambda x, y: x.join(y)), dfs)#; print(df4.shape)
    repeats = round(HIGEST_RATE/4)#; print('repeats =', repeats)
    df4 = df4.loc[np.repeat(df4.index.values, repeats)]#; print(len(df4))
    df4 = df4.reset_index(drop=True)

    #print('rate=8')
    dfs = get_dfs_for_rate(8, mat)
    df8 = reduce((lambda x, y: x.join(y)), dfs)#; print(df8.shape)
    repeats = round(HIGEST_RATE/8)#; print('repeats =', repeats)
    df8 = df8.loc[np.repeat(df8.index.values, repeats)]#; print(len(df8))
    df8 = df8.reset_index(drop=True)

    #print('rate=16')
    dfs = get_dfs_for_rate(16, mat)
    df16 = reduce((lambda x, y: x.join(y)), dfs)#; print(df16.shape)
    repeats = round(HIGEST_RATE/16)#; print('repeats =', repeats)
    #df16 = df16.loc[np.repeat(df16.index.values, repeats)]; print(len(df16))
    #df16 = df16.reset_index(drop=True)

    # Combine all sets of rate dataframes
    df = dfp25.join(df1).join(df2).join(df4).join(df8).join(df16); print('combined=', df.shape)

    # Remove invalid time values
    df2 = df.loc[(df.DATE_YEAR>1990)&(df.DATE_YEAR<2020)& \
             (df.DATE_MONTH>0)&(df.DATE_MONTH<13)& \
             (df.DATE_DAY>0)&(df.DATE_DAY<32)& \
             (df.GMT_HOUR>-1)&(df.GMT_HOUR<24)& \
             (df.GMT_MINUTE>-1)&(df.GMT_MINUTE<60)& \
             (df.GMT_SEC>-1)&(df.GMT_SEC<60)]; print('invalid times removed=', df2.shape)
    if df2.shape[0]==0:
        print('!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!')
        prts = file_.split("/"); p0 = prts[0]; p1 = prts[1]
        !rm {p0}/hold/{p1}
        !rm {file_} #origs are kept in 'holder'
        print(f'{p0}/hold/{p1} deleted')
        print(f'{file_} deleted')
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n')
        continue
        #exit()
        
    # Write to .csv file so that we can use read_csv to parse index
    csv_name = str(file_).replace('.mat', '.csv')
    parts = csv_name.split('.'); proper = parts[0]; ext = parts[1]
    df2.to_csv(csv_name)#; print('csv_name=', csv_name)

    # Build index
    df = pd.read_csv(csv_name, 
                     parse_dates={'datetime': ['DATE_YEAR','DATE_MONTH','DATE_DAY','GMT_HOUR','GMT_MINUTE','GMT_SEC']},
                     index_col='datetime', 
                     low_memory=False)#; print('build index=', df.shape)#; print(df.head())

    #change index dtype from string to datetime
    df.index = [datetime.strptime(s, '%Y %m %d %H %M %S') for s in df.index]
    df.index.name = 'datetime'
    df = df.drop(['Unnamed: 0'], axis=1)#; print('change to datetime=', df.shape)#; print(df.head())
    
    #write full data in case we want to experiment with it
    #df.to_csv(f'{TAIL}/{MATFILENAME}-full.csv')

    # Down-sample to 1 minute rate
    df = df.resample('1min').mean(); print('down-sample=', df.shape)#; print(df.head()) #minutes
    #df = df.resample('1S').interpolate()#; print(df.shape); print(df.head()) #seconds
    df.to_csv(f'{proper}-1min.csv')
    !rm {file_} #origs are kept in 'hold'
    !rm {csv_name} #tmp file
          
    print(f'{i+1} of {n} complete ........................\n')

TOTAL FILES TO PROCESS: 582

Tail_687_3/687200204021439.mat
combined= (89024, 186)
invalid times removed= (89024, 186)
down-sample= (94, 180)
1 of 582 complete ........................

Tail_687_3/687200202101252.mat
combined= (69824, 186)
invalid times removed= (69480, 186)
down-sample= (74, 180)
2 of 582 complete ........................

Tail_687_3/687200205051401.mat
combined= (46912, 186)
invalid times removed= (46912, 186)
down-sample= (50, 180)
3 of 582 complete ........................

Tail_687_3/687200203130915.mat
combined= (66368, 186)
invalid times removed= (66368, 186)
down-sample= (70, 180)
4 of 582 complete ........................

Tail_687_3/687200201210419.mat
combined= (11904, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201210419.mat deleted
Tail_687_3/687200201210419.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200205010654.mat
combined= (74944, 186)
inv

down-sample= (69, 180)
48 of 582 complete ........................

Tail_687_3/687200202171714.mat
combined= (67520, 186)
invalid times removed= (67520, 186)
down-sample= (72, 180)
49 of 582 complete ........................

Tail_687_3/687200204261800.mat
combined= (90112, 186)
invalid times removed= (90112, 186)
down-sample= (95, 180)
50 of 582 complete ........................

Tail_687_3/687200203070431.mat
combined= (65344, 186)
invalid times removed= (65344, 186)
down-sample= (69, 180)
51 of 582 complete ........................

Tail_687_3/687200201021404.mat
combined= (97344, 186)
invalid times removed= (96728, 186)
down-sample= (103, 180)
52 of 582 complete ........................

Tail_687_3/687200203290814.mat
combined= (59200, 186)
invalid times removed= (59200, 186)
down-sample= (62, 180)
53 of 582 complete ........................

Tail_687_3/687200201221351.mat
combined= (89088, 186)
invalid times removed= (89088, 186)
down-sample= (94, 180)
54 of 582 complete .........

combined= (81984, 186)
invalid times removed= (81376, 186)
down-sample= (86, 180)
97 of 582 complete ........................

Tail_687_3/687200201121113.mat
combined= (45312, 186)
invalid times removed= (45312, 186)
down-sample= (48, 180)
98 of 582 complete ........................

Tail_687_3/687200201260338.mat
combined= (14912, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201260338.mat deleted
Tail_687_3/687200201260338.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200201081501.mat
combined= (193728, 186)
invalid times removed= (193728, 186)
down-sample= (203, 180)
100 of 582 complete ........................

Tail_687_3/687200204271927.mat
combined= (16960, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200204271927.mat deleted
Tail_687_3/687200204271927.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/hold/687200203251912.mat deleted
Tail_687_3/687200203251912.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200201010908.mat
combined= (62784, 186)
invalid times removed= (62784, 186)
down-sample= (66, 180)
144 of 582 complete ........................

Tail_687_3/687200204011718.mat
combined= (139968, 186)
invalid times removed= (138992, 186)
down-sample= (146, 180)
145 of 582 complete ........................

Tail_687_3/687200201201615.mat
combined= (17280, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201201615.mat deleted
Tail_687_3/687200201201615.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200201221901.mat
combined= (9536, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201221901.mat deleted
Tail_687_3/687200201221901.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!

combined= (118848, 186)
invalid times removed= (118848, 186)
down-sample= (125, 180)
187 of 582 complete ........................

Tail_687_3/687200201200038.mat
combined= (1920, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201200038.mat deleted
Tail_687_3/687200201200038.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200203310729.mat
combined= (152640, 186)
invalid times removed= (152392, 186)
down-sample= (160, 180)
189 of 582 complete ........................

Tail_687_3/687200201160946.mat
combined= (127552, 186)
invalid times removed= (126384, 186)
down-sample= (133, 180)
190 of 582 complete ........................

Tail_687_3/687200201021106.mat
combined= (94016, 186)
invalid times removed= (92736, 186)
down-sample= (98, 180)
191 of 582 complete ........................

Tail_687_3/687200202081251.mat
combined= (38592, 186)
invalid times removed= (36040, 186)
down-sample

...

combined= (98176, 186)
invalid times removed= (97200, 186)
down-sample= (102, 180)
377 of 582 complete ........................

Tail_687_3/687200201040640.mat
combined= (14592, 186)
invalid times removed= (14592, 186)
down-sample= (16, 180)
378 of 582 complete ........................

Tail_687_3/687200202171113.mat
combined= (89792, 186)
invalid times removed= (89792, 186)
down-sample= (94, 180)
379 of 582 complete ........................

Tail_687_3/687200203271254.mat
combined= (115200, 186)
invalid times removed= (115200, 186)
down-sample= (121, 180)
380 of 582 complete ........................

Tail_687_3/687200203051629.mat
combined= (69056, 186)
invalid times removed= (69056, 186)
down-sample= (73, 180)
381 of 582 complete ........................

Tail_687_3/687200203131528.mat
combined= (182464, 186)
invalid times removed= (182464, 186)
down-sample= (192, 180)
382 of 582 complete ........................

Tail_687_3/687200201031028.mat
combined= (58560, 186)
invalid times re

combined= (192, 186)
invalid times removed= (192, 186)
down-sample= (1, 180)
426 of 582 complete ........................

Tail_687_3/687200205041604.mat
combined= (92480, 186)
invalid times removed= (92480, 186)
down-sample= (1530, 180)
427 of 582 complete ........................

Tail_687_3/687200205031712.mat
combined= (117184, 186)
invalid times removed= (116816, 186)
down-sample= (1463, 180)
428 of 582 complete ........................

Tail_687_3/687200204030448.mat
combined= (94656, 186)
invalid times removed= (93776, 186)
down-sample= (99, 180)
429 of 582 complete ........................

Tail_687_3/687200203211449.mat
combined= (45760, 186)
invalid times removed= (45760, 186)
down-sample= (48, 180)
430 of 582 complete ........................

Tail_687_3/687200204290709.mat
combined= (52288, 186)
invalid times removed= (52288, 186)
down-sample= (55, 180)
431 of 582 complete ........................

Tail_687_3/687200205090314.mat
combined= (4032, 186)
invalid times removed= 

down-sample= (98, 180)
473 of 582 complete ........................

Tail_687_3/687200203151546.mat
combined= (104256, 186)
invalid times removed= (99496, 186)
down-sample= (105, 180)
474 of 582 complete ........................

Tail_687_3/687200201140746.mat
combined= (62528, 186)
invalid times removed= (62528, 186)
down-sample= (67, 180)
475 of 582 complete ........................

Tail_687_3/687200201121251.mat
combined= (153536, 186)
invalid times removed= (153192, 186)
down-sample= (161, 180)
476 of 582 complete ........................

Tail_687_3/687200201171034.mat
combined= (8832, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201171034.mat deleted
Tail_687_3/687200201171034.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200201281259.mat
combined= (6144, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/68720

down-sample= (123, 180)
519 of 582 complete ........................

Tail_687_3/687200204281733.mat
combined= (97600, 186)
invalid times removed= (96568, 186)
down-sample= (102, 180)
520 of 582 complete ........................

Tail_687_3/687200203142127.mat
combined= (72000, 186)
invalid times removed= (72000, 186)
down-sample= (76, 180)
521 of 582 complete ........................

Tail_687_3/687200201131033.mat
combined= (24640, 186)
invalid times removed= (24640, 186)
down-sample= (27, 180)
522 of 582 complete ........................

Tail_687_3/687200202111254.mat
combined= (116480, 186)
invalid times removed= (115440, 186)
down-sample= (121, 180)
523 of 582 complete ........................

Tail_687_3/687200201130841.mat
combined= (73600, 186)
invalid times removed= (73600, 186)
down-sample= (77, 180)
524 of 582 complete ........................

Tail_687_3/687200202161659.mat
combined= (67840, 186)
invalid times removed= (67256, 186)
down-sample= (1476, 180)
525 of 582 compl

combined= (15552, 186)
invalid times removed= (0, 186)
!!!!!!!!!!!!! DELETING UNUSABLE FILE !!!!!!!!!!!!!!!!!!!!
Tail_687_3/hold/687200201130320.mat deleted
Tail_687_3/687200201130320.mat deleted
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Tail_687_3/687200203161750.mat
combined= (85888, 186)
invalid times removed= (85888, 186)
down-sample= (91, 180)
568 of 582 complete ........................

Tail_687_3/687200205010855.mat
combined= (72704, 186)
invalid times removed= (72448, 186)
down-sample= (76, 180)
569 of 582 complete ........................

Tail_687_3/687200202170527.mat
combined= (39936, 186)
invalid times removed= (39368, 186)
down-sample= (42, 180)
570 of 582 complete ........................

Tail_687_3/687200202100953.mat
combined= (117056, 186)
invalid times removed= (114672, 186)
down-sample= (121, 180)
571 of 582 complete ........................

Tail_687_3/687200202151440.mat
combined= (57536, 186)
invalid times removed= (57536, 186)
down-sample= (61

In [0]:
#mv all output files (.csv) from tail dir to 1min
! ls {TAIL}
! mv {TAIL}/*.csv {TAIL}/1min  

1min			  687200202041529-1min.csv  687200203201713-1min.csv
687200112311614-1min.csv  687200202041706-1min.csv  687200203210420-1min.csv
687200112312147-1min.csv  687200202061155-1min.csv  687200203210750-1min.csv
687200201010631-1min.csv  687200202061438-1min.csv  687200203211026-1min.csv
687200201010908-1min.csv  687200202061705-1min.csv  687200203211357-1min.csv
687200201011045-1min.csv  687200202070425-1min.csv  687200203211449-1min.csv
687200201011305-1min.csv  687200202070802-1min.csv  687200203211556-1min.csv
687200201011535-1min.csv  687200202071011-1min.csv  687200203211750-1min.csv
687200201011743-1min.csv  687200202071302-1min.csv  687200203220533-1min.csv
687200201020650-1min.csv  687200202071534-1min.csv  687200203220753-1min.csv
687200201020851-1min.csv  687200202071826-1min.csv  687200203220943-1min.csv
687200201021106-1min.csv  687200202080707-1min.csv  687200203221312-1min.csv
687200201021404-1min.csv  687200202081137-1min.csv  687200203230830-1min.csv
687200201021619-

In [0]:
! ls {TAIL}

1min  hold  png1


In [0]:
! ls {TAIL}/1min

687200112311614-1min.csv  687200202041529-1min.csv  687200203201331-1min.csv
687200112312147-1min.csv  687200202041706-1min.csv  687200203201713-1min.csv
687200201010631-1min.csv  687200202061155-1min.csv  687200203210420-1min.csv
687200201010908-1min.csv  687200202061438-1min.csv  687200203210750-1min.csv
687200201011045-1min.csv  687200202061705-1min.csv  687200203211026-1min.csv
687200201011305-1min.csv  687200202070425-1min.csv  687200203211357-1min.csv
687200201011535-1min.csv  687200202070802-1min.csv  687200203211449-1min.csv
687200201011743-1min.csv  687200202071011-1min.csv  687200203211556-1min.csv
687200201020650-1min.csv  687200202071302-1min.csv  687200203211750-1min.csv
687200201020851-1min.csv  687200202071534-1min.csv  687200203220533-1min.csv
687200201021106-1min.csv  687200202071826-1min.csv  687200203220753-1min.csv
687200201021404-1min.csv  687200202080707-1min.csv  687200203220943-1min.csv
687200201021619-1min.csv  687200202081137-1min.csv  687200203221312-1min.csv